# TFLITE generation for unity application

In [ ]:
import os
import time
import tempfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.python.platform import gfile
from keras.datasets import cifar10
from keras.optimizers import SGD,Adam
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
from tensorflow.python.client import device_lib

* Device check (whether using cpu or gpu)

In [ ]:
device_lib.list_local_devices()

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# print(gpus)

In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

-----------

## Current Project summary

* current model is coco_ssd_mobilenet
* (https://github.com/asus4/tf-lite-unity-sample)
* It follows object detection supported from tensorflow
* (https://www.tensorflow.org/lite/models/object_detection/overview)

-------

## Training dataset load

* Here, we are going to use CIFAR-10 dataset for simple testing

In [ ]:
# Load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

In [ ]:
#Print the dimensions of the datasets

print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

In [ ]:
# Normalizing
# x_train=x_train / 255.0
# x_test=x_test / 255.0

x_train=x_train.astype(np.float32) / 255.0
x_test=x_test.astype(np.float32) / 255.0

#One hot encoding
y_train_cat=to_categorical(y_train,10)
y_test_cat=to_categorical(y_test,10)

In [ ]:
## Visualization for sample training dataset image
# img = plt.imshow(x_train[1])

---------------

## Prepare the detection model

* Here, we try transfer learning, starting from pre-trained model "MobileNetV2"

In [ ]:
## importing original MobileNetV2 model
from keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
## Reorganizing the MobileNetV2 model for our own custom dataset
## and be ready for the transfer learning

# base_model = MobileNetV2(weights='imagenet') ## loading original MobileNetV2 model(input size is 224x224x3)

base_model = MobileNetV2(
    input_shape = (32,32,3),         # input image resolution
    alpha = 1.0,                     # float, larger than zero, controls the width of the network
    include_top = False,             # whether to include the fully-connected layer at the top of the network.
    weights= 'imagenet',             # None(random initialization), 'imagenet'(pre-training of ImageNet) or the path to the weights
    input_tensor = None,             # optional keras tensor to use as image input
    pooling = None,                  # optional pooling mode for feature extraction (None, avg, max)
    # classes = y_train.shape[1],      # integer number of classes to classify images into
    # classifier_activation='softmax'  # activation function to use on the "top" layer
)

In [ ]:
## Check the overall architecture of the base model(mobilenetv2)
base_model.summary()

In [ ]:
#Adding layers to base model of MobileNet

model = Sequential()

#Creating base layer
model.add(base_model)
# model.add(Dropout(0.5))
model.add(Flatten())

#Adding the Dense Layers and Dropout
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dropout(0.3))
model.add(Dense(128,activation=('relu')))
model.add(Dropout(0.2))
model.add(Dense(10, activation=('softmax')))

In [ ]:
model.summary()

In [ ]:
#Compiling Model using SGD 

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
## Setting my own Callback system
## Adjusting EarlyStopping supported by keras
AndrewCallback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',           # quantity to be monitored
    # min_delta=0,                  # minimum change in the monitored quantity to qualify as an improvement
    patience=5,                   # number of epochs with no improvement
    verbose=1,                    # 0 is silent, 1 displays messages
    mode='auto',                  # auto / min: will stop monitored has stopped decreasing / max: stop when monitored has stopped increasing
    # baseline=None,                # baseline value
    restore_best_weights=True,    # whether to restore model weights from the epoch with the best value of the monitored quantity
    # start_from_epoch=0            # numberof epochs to wait before starting to monitor improvement
)

In [ ]:
#Training Model
with tf.device('/device:GPU:0'):
    hist = model.fit(x_train, y_train_cat,      # x, y dataset for model training
                     batch_size = 100,          # model training batch size
                     epochs = 20,               # model training epochs
                     validation_split = 0.1,    # setting validation ratio
                     # callbacks= AndrewCallback  # callback settings (for early stopping, ...)
                    )

In [ ]:
#Testing accuracy of trained model
with tf.device('/device:GPU:0'):
    model.evaluate(x_test, y_test_cat)[1]

In [ ]:
## Visualizing Model Accuracy
fig = plt.figure(figsize=(4,3))
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc = 'upper left')

-----------

## Saving the model

In [ ]:
## check the current model path (where pb file saved)
model.save('C:/Users/user/Jupyter/ML_practices/Tensorflow_prac/SSD_object_detection/retrained_model') #path and name

----------

## Converting the pb model into tflite file

In [ ]:
## load pb model
model_path = 'C:/Users/user/Jupyter/ML_practices/Tensorflow_prac/SSD_object_detection/retrained_model/'
saved_model = tf.saved_model.load(model_path)

In [ ]:
saved_model

* Setting the converter

In [ ]:
# def representative_data_gen():
#     for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
#         yield [input_value]

In [ ]:
def representative_dataset():
    for _ in range(100):
        data = np.random.rand(1, 32, 32, 3)
        yield [data.astype(np.float32)]

In [ ]:
def representative_dataset():
    for image in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [image]

In [ ]:
## Converter without using representative_dataset(simple)
# converter = tf.lite.TFLiteConverter.from_saved_model(model_path) # path to the SavedModel directory
# converter.optimizations = [tf.lite.Optimize.DEFAULT]

## Converter using representative_dataset
converter = tf.lite.TFLiteConverter.from_saved_model(model_path) # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.float32

* Convert the saved pb model into tflite format

In [ ]:
tflite_model = converter.convert()

* Save the tflite model into the path

In [ ]:
open('C:/Users/user/Jupyter/ML_practices/Tensorflow_prac/SSD_object_detection/retrained_model/coco_ssd_mobilenet_quant.tflite', "wb").write(tflite_model)

----------

# Testing current model

In [ ]:
# current_model_path = 'C:/Users/user/Jupyter/ML_practices/Tensorflow_prac/SSD_object_detection/retrained_model/coco_ssd_mobilenet_quant.tflite'
current_model_path = 'C:/Users/user/Jupyter/ML_practices/Tensorflow_prac/SSD_object_detection/retrained_model/original/ssd_original.tflite'

In [ ]:
interpreter = tf.lite.Interpreter(model_path=current_model_path) ## loading current model(coco_ssd_mobilenet)
interpreter.allocate_tensors() ## tensor initialization

In [ ]:
interpreter

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_details

In [ ]:
output_details

In [ ]:
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

In [ ]:
input_shape

In [ ]:
input_data.shape

In [ ]:
interpreter.set_tensor(input_details[0]['index'], input_data)

In [ ]:
interpreter.invoke()

In [ ]:
output_data = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
print(output_data.shape)

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_content=interpreter, gpu_compatibility=True)